# Exploration des données

In [68]:
# IMPORTS :
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import seaborn as sns
import sys

# FUNCTIONS / CLASSES :
%load_ext autoreload
%autoreload 2
sys.path.append("./code")
from data_processing import WMT22

## DICTIONNAIRE DES COUPLES DE TRADUCTIONS :
sl_tls = {
    "English-German" : "ende",
    "English-Russian" : "enru",
    "Chinese-English" : "zhen"
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Accès aux données 

In [40]:
## PARCOURS DES COUPLES DE TRADUCTION POSSIBLES :
for key in sl_tls.keys() :
    
    val = sl_tls[key]
    
    ## ACCES AUX DONNEES DE TRAIN ET DEV :
    df = WMT22.read_data(sl_tl = val)
    
    ## NOMBRE D'OBSERVATIONS :
    print(key , " : " , len(df) , " observations" )
    


English-German  :  32190  observations
English-Russian  :  32348  observations
Chinese-English  :  35188  observations


In [190]:
sl_tl = "enru"
df_rates = pd.read_table("./data/generalMT2022/" + sl_tl + "/mqm_generalMT2022_" + sl_tl + ".tsv",
                                 on_bad_lines = "skip",
                                 index_col = False).set_index("seg_id")

In [172]:
df.sample(1)

,sys,hyp,domain,doc,source,ref,score
seg_id,,,,,,,
1201,PROMT,"Между тем, в Болгарии проливные дожди и крупны...",news,euronews-en.19225,"Meanwhile, in Bulgaria, heavy rains and major ...",Тем временем в Болгарии за выходные проливные ...,-0.0


In [175]:
df_rates.loc[471]

,system,doc,doc_id,rater,source,target,category,severity
seg_id,,,,,,,,
471,comet_bestmbr,ecommerce_en_38,2,unbabel_rater4,It includes a power amplifier to drive loudspe...,В комплект входит усилитель мощности для громк...,No-error,No-error
471,JDExploreAcademy,ecommerce_en_38,2,unbabel_rater4,It includes a power amplifier to drive loudspe...,Он включает в себя усилитель мощности для упра...,No-error,No-error
471,bleu_bestmbr,ecommerce_en_38,2,unbabel_rater4,It includes a power amplifier to drive loudspe...,Он включает в себя усилитель мощности для упра...,No-error,No-error
471,SRPOL,ecommerce_en_38,2,unbabel_rater4,It includes a power amplifier to drive loudspe...,Он включает в себя усилитель мощности для прив...,No-error,No-error
471,Online-G,ecommerce_en_38,2,unbabel_rater4,It includes a power amplifier to drive loudspe...,Он включает в себя усилитель мощности для упра...,No-error,No-error
471,refA,ecommerce_en_38,2,unbabel_rater4,It includes a power amplifier to drive loudspe...,"Он включает усилитель мощности, чтобы управлят...",No-error,No-error
471,eTranslation,ecommerce_en_38,2,unbabel_rater4,It includes a power amplifier to drive loudspe...,Он включает в себя усилитель мощности для прив...,No-error,No-error
471,Online-B,ecommerce_en_38,2,unbabel_rater4,It includes a power amplifier to drive loudspe...,Он включает в себя усилитель мощности для упра...,No-error,No-error
471,Online-A,ecommerce_en_38,2,unbabel_rater4,It includes a power amplifier to drive loudspe...,Он включает в себя усилитель мощности для прив...,No-error,No-error


In [179]:
df_rates.sample(1)

,system,doc,doc_id,rater,source,target,category,severity
seg_id,,,,,,,,
1580,QUARTZ_TuneReranking,t1_hiyjod1,0,unbabel_rater3,There's a circularity to it...,В этом есть какая-то <v>непоследовательность</...,mistranslation,major


In [191]:
WMT22.MQM_unlabel_score(
                df_rates = df_rates,
                seg_id = 471,
                sys = "comet_bestmbr"
                )

100

In [114]:
WMT22.MQM_google_score(
                df_rates = df_rates,
                seg_id = 1,
                sys = "QUARTZ_TuneReranking"
                )

-1

In [55]:
res

'Online-B'

In [194]:
df = WMT22.read_data(sl_tl = "enru")

In [195]:
df

,sys,hyp,domain,doc,source,ref,score,score_test
seg_id,,,,,,,,
1,bleu_bestmbr,Я постоянно вижу это в своей работе.,social,t1_hqh0ref,I see it all the time in my line of work.,Я постоянно вижу такое в своей сфере деятельно...,-0.0,100
2,bleu_bestmbr,"И даже не обязательно вопрос жизни или смерти,...",social,t1_hqh0ref,And it doesn’t even have to be life or death t...,И это даже не должно быть вопросом жизни и сме...,-5.0,-400
3,bleu_bestmbr,"Недавно был пациент, которому потребовалась оч...",social,t1_hqh0ref,Had a recent patient who needed a very specifi...,"Недавно у меня был пациент, которому была необ...",-0.0,100
4,bleu_bestmbr,Одноранговый обмен не удалось.,social,t1_hqh0ref,Peer to peer failed.,Ошибка одноранговой сети.,-15.0,-1400
5,bleu_bestmbr,"Страховка говорит, что это не нужно по медицин...",social,t1_hqh0ref,Insurance says it’s not medically necessary.,"В страховке говорится, что это не является нео...",-0.0,100
...,...,...,...,...,...,...,...,...
2033,SRPOL,"Пожалуйста, заполните его, если у вас есть мом...",conversation,en_de_CLIENT-01_default_2021-02-14-9_doc0,Please kindly fill this in if you have a momen...,"Пожалуйста, заполните ее, если у Вас есть своб...",-2.0,-300
2034,SRPOL,Айова Санта уходит на пенсию после 50 лет,news,news.com.au.33709,Iowa Santa retires after 50 years,Санта-Клаус штата Айова ушел в отставку спустя...,-5.0,-400
2035,SRPOL,"Айова Санта, который заставляет детей улыбатьс...",news,news.com.au.33709,An Iowa Santa who has been making children smi...,"В штате Айова Санта-Клаус, даривший детям улыб...",-6.0,-500
